<a href="https://colab.research.google.com/github/mightyoctopus/lora-fine-tuning-example-code/blob/main/LoRA_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q peft transformers datasets torch accelerate

In [ ]:
!pip install -U bitsandbytes

Dataset: Rabe3/QA_Synthatic_Medical_data

Model (Instrcut): Qwen/Qwen3-0.6B

- Load model
- tokenizing
- quantization
- load dataset
- Prepare Dataset Format

tokenizer.apply_chat_template(messages, tokenize=False)

- Lora Config
- train
- Save the fine tuned model

In [8]:
from torch.utils.data import DataLoader
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
                          )
from peft import LoraConfig, get_peft_model, TaskType

from google.colab import drive, userdata
from huggingface_hub import login, snapshot_download

import os

In [4]:
model_name = "Qwen/Qwen3-0.6B"
dataset_name = "Rabe3/QA_Synthatic_Medical_data"

In [ ]:
drive.mount("/content/drive")
cache_path = "/content/drive/My Drive/models/huggingface_cache"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model_folder_name = "models--" + model_name.replace("/", "--")
parent_model_path_in_drive = os.path.join(cache_path, model_folder_name)

if not os.path.exists(parent_model_path_in_drive):
    print("Model not found in Drive -- downloading from HF...")
    model_path = snapshot_download(
        repo_id=model_name,
        cache_dir=cache_path,
        local_dir_use_symlinks=False
    )
else:
    print("Model found in Drive -- fetching from the cache...")
    snapshots_dir = os.path.join(parent_model_path_in_drive, "snapshots")
    drive_id = os.listdir(snapshots_dir)

    if drive_id:
        model_path = os.path.join(snapshots_dir, drive_id[0])
    else:
        raise ValueError("No snapshot found in the cache path.")

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    cache_dir=cache_path
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)


In [10]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
data = load_dataset(dataset_name, "default", split="train[:200]")

In [12]:
def tokenize(batch):

    for convo in batch["conversations"]:
        for turn in convo:
            human_msg = turn["value"] if turn["from"] == "human" else ""
            assisant_msg = turn["value"] if turn["from"] == "gpt" else ""

            texts = [
                f"### Instruction:\n{human_msg}\n### Response:\n{assisant_msg}"
            ]

    tokens = tokenizer(
        texts,
        padding="max_length",
        max_length=256,
        truncation=True,
        return_tensors="pt"
    )

    tokens["labels"] = tokens["input_ids"].clone()

    return tokens

In [ ]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

In [14]:
training_args = TrainingArguments(
    output_dir = "./fine_tuned_result",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate= 1e-3,
    num_train_epochs=50,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none"
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    processing_class=tokenizer
)

In [ ]:
result = trainer.train()

In [ ]:
print(result.training_loss)

In [ ]:
### Get Perplexity:
import math

loss = result.training_loss
perplexity = math.exp(loss)

print(f"Perplexity: {perplexity}")